<a href="https://colab.research.google.com/github/wandb/examples/blob/master/colabs/pytorch/Artifacts_Basics.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>
<!--- @wandbcode{artifacts-basics} -->

<img src="http://wandb.me/logo-im-png" width="400" alt="Weights & Biases" />

<!--- @wandbcode{pytorch-video} -->



Use [Weights & Biases](https://wandb.com) for machine learning experiment tracking, dataset versioning, and project collaboration.

<div><img /></div>

<img src="https://wandb.me/mini-diagram" width="650" alt="Weights & Biases" />

<div><img /></div>




Use W&B Artifacts to track and version data as the inputs and outputs of your W&B Runs. For example, a model training run might take in a dataset as input and trained model as output. In addition to logging hyperparameters and metadata to a run, you can use an artifact to log the dataset used to train the model as input and the resulting model checkpoints as outputs.

# Set Up

In order to log data to our web service, you need to log in and import `wandb`. If this is your first time using W&B, you'll need to sign up for a free account at the link that appears.

In [ ]:
!pip install wandb
import wandb
wandb.login()

# Create An Artifact

The general workflow for creating an Artifact is:


1.   Intialize a run.
2.   Create an Artifact.
3.   Add a dataset, a model, or another Artifact as an input.
4.   Log the artifact in the W&B platform.

This can by accomplished in two lines of code:

In [ ]:
run = wandb.init(project="artifact-basics")
run.log_artifact(artifact_or_path="/content/sample_data/mnist_test.csv", name="my_first_artifact", type="dataset")
run.finish()

First, initalize the run with [`wandb.init()`](https://docs.wandb.ai/ref/python/init). In this demo, the code adds the run to the `artifact-basic` project, but you can change the name to anyting you'd like.

Next, log the artifact with [`run.log_artifact()`](https://docs.wandb.ai/ref/python/public-api/run#log_artifact). In this demo, the artifact is a `dataset` using data from `mnist_test.csv`. You can customize your artifact with a `name` and other metadata- see the Artifacts Reference guide for more information.


If you change or add any parameters names, be sure to replicate those changes in the following code samples as well.

# Use an Artifact

When you need to select a particular Artifact, indicate the artifact you want to use with the [`use_artifact`](https://docs.wandb.ai/ref/python/public-api/run#use_artifact) method. This returns a [run](https://docs.wandb.ai/ref/python/public-api/run) object and marks the artifact version as an input to the run. In the proceeding code snippet specifies an artifact called `my_first_artifact` with the alias `latest`:


In [ ]:
run = wandb.init(project="artifact-basics")
artifact = run.use_artifact(artifact_or_name="my_first_artifact:latest")
run.finish()

# Create a new Artifact version

When you need add a new file, you can use the [`artifact.add_file`](https://docs.wandb.ai/ref/python/artifact#add_file) method.

In [ ]:
run = wandb.init(project="artifact-basics")
artifact = run.use_artifact(artifact_or_name="my_first_artifact:latest") # selects the artifact you're adding the file to
artifact.add_file(local_path="/content/sample_data/california_housing_test.csv", name="new_file")
run.log_artifact(artifact)
run.finish()

This adds a new .csv file called `new_file` to the `my_first_artifact` Artifact. 

Optionally, you can choose to add an entire directory to an Artifact:

In [ ]:
run = wandb.init(project="artifact-basics")
artifact = run.use_artifact(artifact_or_name="my_first_artifact:latest") # selects the artifact you're adding the file to, and makes a new
artifact.add_dir(local_path="/content/sample_data", name="new_directory")
run.log_artifact(artifact, aliases= ["directory_version"])
run.finish()

This adds a new version of `my_first_artifact` with an entire folder with several .csv files. To save your changes, you'll need to [`log`](https://docs.wandb.ai/ref/python/log) or [`save`](https://docs.wandb.ai/ref/python/artifact#save) them.

# Update an Artifact

You can pass new values to update the `description`, `metadata`, and `alias` of an artifact. Call the `save()` method to update the artifact on the W&B servers. You can update an artifact during or outside a W&B Run.


This example changes the `description` of the `my_first_artifact` artifact inside a run:

In [ ]:
run = wandb.init(project="artifact-basics")
artifact = run.use_artifact(artifact_or_name="my_first_artifact:latest")
artifact.description = "This is an edited description."
artifact.save()  # persists changes to an Artifact's properties
run.finish()

# Download an Artifact

To retrieve the path of an Artifact for external use, use the [`artifact.download()`](https://docs.wandb.ai/ref/python/artifact#download) method. This will retrieve the directory of an Artifact you select.

In [ ]:
run = wandb.init(project="artifact-basics")
artifact = run.use_artifact(artifact_or_name="my_first_artifact:latest")
# downloads the artifact to a local directory.
datadir = artifact.download()
run.finish()
# prints the path of the current artifact directory
print(u'\u2500' * 10)
print("Data directory located at:" + datadir)

For more information on ways to customize your Artifact download, see the [Download and Usage guide](https://docs.wandb.ai/guides/artifacts/download-and-use-an-artifact).

# Navigate the Artifacts UI

You can also manage your Artifacts via the W&B platform. This can give you insight into your model's performance or dataset versioning. To navigate to the relevant information, click this [link](https://wandb.ai/wandb/artifact-basics/overview), then click on the **Artifacts** tab.

Navigating to the **Lineage** section in the tab will show the dependency graph formed by calling `run.use_artifact()` when an Artifact is an input to a run, and `run.log_artifact()` when an Artifact is output to a run. This helps visualize the relationship between different model versions and other objects like datasets and jobs in your project. Click [this](https://wandb.ai/wandb/artifact-basics/artifacts/dataset/my_first_artifact/v0/lineage) link to navigate to the project's lineage page.

# Next steps
1. [Artifacts Python reference documentation](https://docs.wandb.ai/ref/python/artifact): Deep dive into artifact parameters and advanced methods.
2. [Lineage](https://docs.wandb.ai/guides/artifacts/explore-and-traverse-an-artifact-graph): View lineage graphs, which show artifacts logged by a run as well as artifacts used by specific run. 
3. [Model Registry](https://docs.wandb.ai/guides/model_registry): Learn how to centralize your best artifact versions in a shared registry.
4. [Artifact Automations](https://docs.wandb.ai/guides/artifacts/project-scoped-automations): Automate downstream actions for versioning artifacts.
5. [Artifacts FAQ](https://docs.wandb.ai/guides/artifacts/artifacts-faqs): Frequently asked questions on Artifact security, workflows, and retention policy.